In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import cv2
import os
import math
import sys
import numpy as np
import pytesseract
import io 
import csv
import pandas as pd
import shutil, os
import re
import warnings
warnings.filterwarnings('ignore')
path = '/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells/input'
for file_name in os.listdir(path):
    
    #print(file_name)
    shutil.copytree('/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells/sample', 
        os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells',
                                     os.path.splitext(file_name)[0]))
    
    def extract_cell_images_from_table(image):
        BLUR_KERNEL_SIZE = (17, 17)
        STD_DEV_X_DIRECTION = 0
        STD_DEV_Y_DIRECTION = 0
        blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~blurred,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )
        vertical = horizontal = img_bin.copy()
        SCALE = 5
        image_width, image_height = horizontal.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
        horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
        vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)

        horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
        vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))

        mask = horizontally_dilated + vertically_dilated
        contours, heirarchy = cv2.findContours(
            mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
        )

        perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
        epsilons = [0.05 * p for p in perimeter_lengths]
        approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]

        # Filter out contours that aren't rectangular. Those that aren't rectangular
        # are probably noise.
        approx_rects = [p for p in approx_polys if len(p) == 4]
        bounding_rects = [cv2.boundingRect(a) for a in approx_polys]

        # Filter out rectangles that are too narrow or too short.
        MIN_RECT_WIDTH = 40
        MIN_RECT_HEIGHT = 10
        bounding_rects = [
            r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
        ]

        # The largest bounding rectangle is assumed to be the entire table.
        # Remove it from the list. We don't want to accidentally try to OCR
        # the entire table.
        largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
        bounding_rects = [b for b in bounding_rects if b is not largest_rect]

        cells = [c for c in bounding_rects]
        def cell_in_same_row(c1, c2):
            c1_center = c1[1] + c1[3] - c1[3] / 2
            c2_bottom = c2[1] + c2[3]
            c2_top = c2[1]
            return c2_top < c1_center < c2_bottom

        orig_cells = [c for c in cells]
        rows = []
        while cells:
            first = cells[0]
            rest = cells[1:]
            cells_in_same_row = sorted(
                [
                    c for c in rest
                    if cell_in_same_row(c, first)
                ],
                key=lambda c: c[0]
            )

            row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
            rows.append(row_cells)
            cells = [
                c for c in rest
                if not cell_in_same_row(c, first)
            ]

        # Sort rows by average height of their center.
        def avg_height_of_center(row):
            centers = [y + h - h / 2 for x, y, w, h in row]
            return sum(centers) / len(centers)

        rows.sort(key=avg_height_of_center)
        cell_images_rows = []
        for row in rows:
            cell_images_row = []
            for x, y, w, h in row:
                cell_images_row.append(image[y:y+h, x:x+w])
            cell_images_rows.append(cell_images_row)
        return cell_images_rows

    def main(f):
        results = []
        directory, filename = os.path.split(f)
        table = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        rows = extract_cell_images_from_table(table)
        #cell_img_dir = os.path.join(directory, "cells")
        #os.makedirs(cell_img_dir, exist_ok=True)
        out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells',
                                    os.path.splitext(file_name)[0])
        paths = []
        for i, row in enumerate(rows):
            for j, cell in enumerate(row):
                cell_filename = "{:03d}-{:03d}.png".format(i, j)
                path = os.path.join(out_path, cell_filename)
                cv2.imwrite(path, cell)
                paths.append(path)
        return paths
    f = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells/input',
                               os.path.splitext(file_name)[0]+'.tif')
    main(f)
    
    text = []
    path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells',
                                    os.path.splitext(file_name)[0])
    file_path = sorted(os.listdir(path))
    for x in file_path:
        txt = pytesseract.image_to_string(os.path.join(path,x))
        text.append(txt)
    #print(text)
    word = 'TOTAL WAGES PAID'
    a = []
    for i in text:
        if fuzz.ratio(word,i) > 77:
            #print(i)
            a.append(i)
            #print(text.index(i))
            ind = text.index(i)
            out = text[ind+1]
            #print(out)
            num_1 = re.sub("[ ,.#*!@$%^&()|\''"":;-_+]", "", out)
            #print(num_1)
            op = list(num_1)
            if op[0].isdigit():
                print(file_name,'TOTAL WAGES PAID',num_1)
            else:
                print(file_name,'TOTAL WAGES PAID','00')
        
    if len(a) == 0:
        print(file_name,'TOTAL WAGES PAID','000')

In [47]:
    import matplotlib.image as mpimg
    import matplotlib.pyplot as plt
    from PIL import Image
    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process
    import cv2
    import os
    import math
    import sys
    import numpy as np
    import pytesseract
    import io 
    import csv
    import pandas as pd
    import shutil, os
    import warnings
    warnings.filterwarnings('ignore')
    def extract_cell_images_from_table(image):
        BLUR_KERNEL_SIZE = (17, 17)
        STD_DEV_X_DIRECTION = 0
        STD_DEV_Y_DIRECTION = 0
        blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~blurred,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )
        vertical = horizontal = img_bin.copy()
        SCALE = 5
        image_width, image_height = horizontal.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
        horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
        vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)

        horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
        vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))

        mask = horizontally_dilated + vertically_dilated
        contours, heirarchy = cv2.findContours(
            mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
        )

        perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
        epsilons = [0.05 * p for p in perimeter_lengths]
        approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]

        # Filter out contours that aren't rectangular. Those that aren't rectangular
        # are probably noise.
        approx_rects = [p for p in approx_polys if len(p) == 4]
        bounding_rects = [cv2.boundingRect(a) for a in approx_polys]

        # Filter out rectangles that are too narrow or too short.
        MIN_RECT_WIDTH = 40
        MIN_RECT_HEIGHT = 10
        bounding_rects = [
            r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
        ]

        # The largest bounding rectangle is assumed to be the entire table.
        # Remove it from the list. We don't want to accidentally try to OCR
        # the entire table.
        largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
        bounding_rects = [b for b in bounding_rects if b is not largest_rect]

        cells = [c for c in bounding_rects]
        def cell_in_same_row(c1, c2):
            c1_center = c1[1] + c1[3] - c1[3] / 2
            c2_bottom = c2[1] + c2[3]
            c2_top = c2[1]
            return c2_top < c1_center < c2_bottom

        orig_cells = [c for c in cells]
        rows = []
        while cells:
            first = cells[0]
            rest = cells[1:]
            cells_in_same_row = sorted(
                [
                    c for c in rest
                    if cell_in_same_row(c, first)
                ],
                key=lambda c: c[0]
            )

            row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
            rows.append(row_cells)
            cells = [
                c for c in rest
                if not cell_in_same_row(c, first)
            ]

        # Sort rows by average height of their center.
        def avg_height_of_center(row):
            centers = [y + h - h / 2 for x, y, w, h in row]
            return sum(centers) / len(centers)

        rows.sort(key=avg_height_of_center)
        cell_images_rows = []
        for row in rows:
            cell_images_row = []
            for x, y, w, h in row:
                cell_images_row.append(image[y:y+h, x:x+w])
            cell_images_rows.append(cell_images_row)
        return cell_images_rows

    def main(f):
        results = []
        directory, filename = os.path.split(f)
        table = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        rows = extract_cell_images_from_table(table)
        #cell_img_dir = os.path.join(directory, "cells")
        #os.makedirs(cell_img_dir, exist_ok=True)
        out_path = os.path.join('/home/vimalkumar.s/Documents/file samples/test/1st page/cells')
        paths = []
        for i, row in enumerate(rows):
            for j, cell in enumerate(row):
                cell_filename = "{:03d}-{:03d}.png".format(i, j)
                path = os.path.join(out_path, cell_filename)
                cv2.imwrite(path, cell)
                paths.append(path)
        return paths
    f = '/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells/input/0416674000.tif'
    main(f)

['/home/vimalkumar.s/Documents/file samples/test/1st page/cells/000-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/000-001.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/001-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/001-001.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/002-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/002-001.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/003-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/003-001.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/003-002.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/004-000.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/004-001.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/004-002.png',
 '/home/vimalkumar.s/Documents/file samples/test/1st page/cells/

In [48]:
text = []
path = os.path.join('/home/vimalkumar.s/Documents/file samples/test/1st page/cells')
                                    
file_path = sorted(os.listdir(path))
for x in file_path:
    txt = pytesseract.image_to_string(os.path.join(path,x))
    text.append(txt)
print(text)

[' \n\n04-16674-0-00~-. . - : -\n\x0c', '\x0c', '( (WRAH.2021\n‘ard Ath\n\n \n\n \n\x0c', 'Date Paid\n\x0c', ' \n\x0c', '\x0c', ' \n\x0c', '‘78,548 00\n\x0c', '\x0c', ' \n\x0c', '22,200 OC\n\x0c', '\x0c', ' \n\x0c', '» 56,348 00\n\x0c', '\x0c', ' \n\x0c', '644.62\n\x0c', '\x0c', ' \n\x0c', '\x0c', '\x0c', '\x0c', '3\n2\n\n \n\nHi\n\n   \n\n3\n\ni\n\n_dofiorsan :\n\ncic oti ne eon ayo 4\nay on at wv untoactiabe re. gov\noe 573-751-1905, wee ee\n\x0c', ' \n\x0c', ' \n\x0c', '\x0c', '\x0c', '\x0c', '49076-7607\n\x0c', 'Jesse\n\x0c', ' \n\n17,\nFirst\nName\n\x0c', '\x0c', '\x0c', '\x0c', 'Delaney\n\x0c', 'Antheny\n\n- Mitehell\n\x0c', 'Wendi\n\x0c', ' \n\nTOTAL THIS PAGE\n\x0c', '\x0c', '\x0c', '\x0c', '\x0c', '\x0c', '\x0c', ' \n\x0c', 'Middle\nTaltial\n\x0c', '\x0c', '\x0c', '\x0c', 'Patrick\n\x0c', '\x0c', '\x0c', '\x0c', 'Last\n\nName\n\x0c', ' \n\x0c', '\x0c', '\x0c', '\x0c', 'sea |\n— |\n|\n\n2,228,39\n\n \n\x0c', '5,985.30\n\x0c', '\x0c', '\x0c', '\x0c', '\x0c', '\x0c', '\x0c', '\x0

In [41]:
str1 = '4, TOTAL WAGES PAID\n\n‘5, WAGES PAID IN EXCESS ¢ OF\n11500.00 PER WORKER\nPER YEAR (See Instruction Sheet)\n\n \n\n \n\nfi TAXABLE WAGES\n— (ter 4 Minus Item 5) '

In [42]:
str2 = 'TOTAL WAGES PAID'
fuzz.ratio(str1,str2)

19

In [52]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import cv2
import os
import math
import sys
import numpy as np
import pytesseract
import io 
import csv
import pandas as pd
import shutil, os
import warnings
warnings.filterwarnings('ignore')
path = '/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells/input'
for file_name in os.listdir(path):
    
    #print(file_name)
    shutil.copytree('/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells/sample', 
        os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells',
                                     os.path.splitext(file_name)[0]))
    
    def extract_cell_images_from_table(image):
        BLUR_KERNEL_SIZE = (17, 17)
        STD_DEV_X_DIRECTION = 0
        STD_DEV_Y_DIRECTION = 0
        blurred = cv2.GaussianBlur(image, BLUR_KERNEL_SIZE, STD_DEV_X_DIRECTION, STD_DEV_Y_DIRECTION)
        MAX_COLOR_VAL = 255
        BLOCK_SIZE = 15
        SUBTRACT_FROM_MEAN = -2

        img_bin = cv2.adaptiveThreshold(
            ~blurred,
            MAX_COLOR_VAL,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY,
            BLOCK_SIZE,
            SUBTRACT_FROM_MEAN,
        )
        vertical = horizontal = img_bin.copy()
        SCALE = 5
        image_width, image_height = horizontal.shape
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(image_width / SCALE), 1))
        horizontally_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, int(image_height / SCALE)))
        vertically_opened = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel)

        horizontally_dilated = cv2.dilate(horizontally_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)))
        vertically_dilated = cv2.dilate(vertically_opened, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 60)))

        mask = horizontally_dilated + vertically_dilated
        contours, heirarchy = cv2.findContours(
            mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE,
        )

        perimeter_lengths = [cv2.arcLength(c, True) for c in contours]
        epsilons = [0.05 * p for p in perimeter_lengths]
        approx_polys = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]

        # Filter out contours that aren't rectangular. Those that aren't rectangular
        # are probably noise.
        approx_rects = [p for p in approx_polys if len(p) == 4]
        bounding_rects = [cv2.boundingRect(a) for a in approx_polys]

        # Filter out rectangles that are too narrow or too short.
        MIN_RECT_WIDTH = 40
        MIN_RECT_HEIGHT = 10
        bounding_rects = [
            r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]
        ]

        # The largest bounding rectangle is assumed to be the entire table.
        # Remove it from the list. We don't want to accidentally try to OCR
        # the entire table.
        largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
        bounding_rects = [b for b in bounding_rects if b is not largest_rect]

        cells = [c for c in bounding_rects]
        def cell_in_same_row(c1, c2):
            c1_center = c1[1] + c1[3] - c1[3] / 2
            c2_bottom = c2[1] + c2[3]
            c2_top = c2[1]
            return c2_top < c1_center < c2_bottom

        orig_cells = [c for c in cells]
        rows = []
        while cells:
            first = cells[0]
            rest = cells[1:]
            cells_in_same_row = sorted(
                [
                    c for c in rest
                    if cell_in_same_row(c, first)
                ],
                key=lambda c: c[0]
            )

            row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
            rows.append(row_cells)
            cells = [
                c for c in rest
                if not cell_in_same_row(c, first)
            ]

        # Sort rows by average height of their center.
        def avg_height_of_center(row):
            centers = [y + h - h / 2 for x, y, w, h in row]
            return sum(centers) / len(centers)

        rows.sort(key=avg_height_of_center)
        cell_images_rows = []
        for row in rows:
            cell_images_row = []
            for x, y, w, h in row:
                cell_images_row.append(image[y:y+h, x:x+w])
            cell_images_rows.append(cell_images_row)
        return cell_images_rows

    def main(f):
        results = []
        directory, filename = os.path.split(f)
        table = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        rows = extract_cell_images_from_table(table)
        #cell_img_dir = os.path.join(directory, "cells")
        #os.makedirs(cell_img_dir, exist_ok=True)
        out_path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells',
                                    os.path.splitext(file_name)[0])
        paths = []
        for i, row in enumerate(rows):
            for j, cell in enumerate(row):
                cell_filename = "{:03d}-{:03d}.png".format(i, j)
                path = os.path.join(out_path, cell_filename)
                cv2.imwrite(path, cell)
                paths.append(path)
        return paths
    f = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells/input',
                               os.path.splitext(file_name)[0]+'.tif')
    main(f)
    
    text = []
    path = os.path.join('/home/vimalkumar.s/Documents/extraction/header/total wages paid/cells',
                                    os.path.splitext(file_name)[0])
    file_path = sorted(os.listdir(path))
    for x in file_path:
        txt = pytesseract.image_to_string(os.path.join(path,x))
        text.append(txt)
    #print(text)
    word = 'TOTAL WAGES PAID'
    d = []
    for i in text:
        #print(i)
        if fuzz.ratio(word,i) > 75:
            #print(i)
            d.append(i)
            #print(text.index(i))
            #print(text[text.index(i)+1])
            val = text[text.index(i)+1]
            f_out = re.sub("[ ,.#*!@$%^&()|\''"":;-]", "",val)
            #print(f_out)
            c = re.findall('[0-9]+',f_out)
            #print(c)
            if c[0].isdigit():
                print(file_name,'TOTAL WAGES PAID',f_out)
            else:
                print(file_name,'TOTAL WAGES PAID','000')
    if len(d) == 0:
        print(file_name,'TOTAL WAGES PAID','00')

    


0450140000.tif TOTAL WAGES PAID 00
0905808000.tif TOTAL WAGES PAID 980213

0462378000.tif TOTAL WAGES PAID 6333830

0911520000.tif TOTAL WAGES PAID 3954989

0313471000.tif TOTAL WAGES PAID 1466625

0356369000.tif TOTAL WAGES PAID 4193025

0300168000.tif TOTAL WAGES PAID 00
0771062000.tif TOTAL WAGES PAID 876201

0560522000.tif TOTAL WAGES PAID 28359226

0308511000.tif TOTAL WAGES PAID 1267500

0719339000.tif TOTAL WAGES PAID 87000

0954699000.tif TOTAL WAGES PAID 174250

0632871000.tif TOTAL WAGES PAID 7926648

0241904000.tif TOTAL WAGES PAID 00
0327919000.tif TOTAL WAGES PAID 372500

0809963000.tif TOTAL WAGES PAID 3966315

0352450000.tif TOTAL WAGES PAID 00
0878579000.tif TOTAL WAGES PAID 88000

0602441000.tif TOTAL WAGES PAID 5WAGESPAIDINEXCESSOF
PERWORKER
PERYEARSeeInstructionSheet

0489279000.tif TOTAL WAGES PAID 1096814

0644730000.tif TOTAL WAGES PAID 2281796

0222259000.tif TOTAL WAGES PAID 789494

0902476000.tif TOTAL WAGES PAID 11790266

0784699000.tif TOTA

IndexError: list index out of range

In [53]:
c

[]

In [54]:
f_out

'\n\x0c'

In [55]:
val

' \n\x0c'

In [56]:
d

['4 TOTAL WAGES PAID\n\x0c']